## LIBRARIESImporting Libraries for data handling, visualising, ML and text analysis

In [ ]:
import pandas as pd # Data manipulation and analysisimport numpy as np # Numerical operationsimport matplotlib.pyplot as plt # Static visualizationsimport seaborn as sns # Enhanced static visualizationsimport plotly.express as px # Interactive visualizationsfrom datetime import datetime# Machine learning and text processing:from sklearn.model_selection import train_test_split, GridSearchCVfrom sklearn.feature_extraction.text import TfidfVectorizerfrom sklearn.linear_model import LogisticRegressionfrom sklearn.svm import SVCfrom sklearn.ensemble import RandomForestClassifierfrom sklearn.ensemble import HistGradientBoostingClassifierfrom sklearn.naive_bayes import MultinomialNBfrom sklearn.metrics import classification_report, confusion_matrixfrom sklearn.pipeline import Pipeline# For generating a word cloud visualization:from wordcloud import WordCloud# For additional text analysis:from collections import Counterimport re# Enable inline plotting (for Jupyter Notebook users)%matplotlib inline

## LOADING & INSPECTING DATA

In [ ]:
# Load the CSV file into a DataFrame. Adjust the path as needed.df = pd.read_csv('/Users/manusokane/Desktop/University/UU Computer Science/COM771 - Cyber Security/RT_IOT2022.csv', encoding='utf-8')# Strip extra whitespace from column names in case there are any hidden spaces.df.columns = df.columns.str.strip()# Display the first few rows to understand the data structure.df.head()# Print dataset info: list of columns, number of non-null values, and data types.df.info()

## DATA CLEANING & PREPROCESSINGNo NaN or NA values in this datasetcritical fields??

In [ ]:
# Createing new DataFramedf_cleaned = df.copy()#Check for missing values in the dataset.missing = df_cleaned.isnull().sum()print(f"There are {missing.sum()} missing values in the dataset.")# Dropping unneccessary columns.df_cleaned.drop(columns=['Unnamed: 0', 'id.orig_p', 'id.resp_p'], inplace=True) df_cleaned.info()

## EXPLORATORY DATA ANALYSIS

In [ ]:
# Distrubtion of the attack typesplt.figure(figsize=(10, 6))sns.countplot(data=df_cleaned, x='Attack_type', order=df_cleaned['Attack_type'].value_counts().index)plt.xlabel("Attack Type")plt.ylabel("Count")plt.title("Distribution of Attack Types")plt.xticks(rotation=45)plt.tight_layout()plt.show()

Bias towards DOS_SYN_HpingCould skew model performance

In [ ]:
# Distrubtion of the attack typesplt.figure(figsize=(10, 6))sns.countplot(data=df_cleaned, x='Attack_type', order=df_cleaned['Attack_type'].value_counts().index)plt.xlabel("Attack Type")plt.ylabel("Count")plt.title("Distribution of Attack Types")plt.xticks(rotation=45)plt.yscale('log')plt.tight_layout()plt.show()

log scaling to represent all classes. results aren't distorted.

In [ ]:
# Protocol distribution as a pie chartprotocol_counts = df_cleaned['proto'].value_counts()protocol_labels = protocol_counts.indexplt.figure(figsize=(10, 6))plt.pie(protocol_counts, labels=protocol_labels, autopct='%1.1f%%', startangle=140)plt.title("Protocol Distribution")plt.tight_layout()plt.show()

Label encoding for categorical columns

In [ ]:
from sklearn.preprocessing import LabelEncoder# Create a copy of the cleaned DataFrame to avoid overwritingdf_encoded = df_cleaned.copy()# Initialize the LabelEncoderlabel_encoder = LabelEncoder()# Identify categorical columnscategorical_columns = df_encoded.select_dtypes(include=['object']).columns# Apply label encoding to each categorical columnfor col in categorical_columns:    df_encoded[col] = label_encoder.fit_transform(df_encoded[col])    print(f"Column '{col}' has been label encoded.")# Display the first few rows of the encoded DataFramedf_encoded.head()

In [ ]:
# Generate the correlation matrixcorrelation_matrix = df_encoded.corr()# Plot the heatmapplt.figure(figsize=(20, 15))sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt='.2f', linewidths=0.5)plt.title("Correlation Matrix Heatmap")plt.tight_layout()plt.show()

In [ ]:
print(df_encoded.head())# Sample a subset for faster processing (e.g., 1000 rows)df_ml_sample = df_encoded.sample(n=min(1000, len(df_encoded)), random_state=42)print("\nShape of the ML sample dataset:", df_ml_sample.shape)

## TRAINING

In [ ]:
from sklearn.model_selection import train_test_splitfrom sklearn.metrics import classification_report, confusion_matrixfrom sklearn.linear_model import LogisticRegressionfrom sklearn.svm import SVCfrom sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifierfrom sklearn.naive_bayes import MultinomialNBimport seaborn as snsimport matplotlib.pyplot as pltX = df_ml_sample.drop(['Attack_type'], axis=1)y = df_ml_sample['Attack_type']X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)# Function to evaluate a modeldef evaluate_model(model, model_name):    # Fit the model    model.fit(X_train, y_train)        # Predict on the test set    y_pred = model.predict(X_test)    # Print model performance report with zero_division set to 0 to handle undefined precision.    print(f"\n--- {model_name} ---")    print(classification_report(y_test, y_pred, zero_division=0))    # Generate and display the confusion matrix    #cm = confusion_matrix(y_test, y_pred)    #plt.figure(figsize=(6, 5))    #sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=model.classes_, yticklabels=model.classes_)    #plt.xlabel("Predicted")    #plt.ylabel("True")    #plt.title(f"Confusion Matrix - {model_name}")    #plt.tight_layout()    #plt.show()# Evaluate different models.evaluate_model(LogisticRegression(max_iter=1000, random_state=42),"Logistic Regression")evaluate_model(SVC(probability=True, random_state=42), "Support Vector Machine")evaluate_model(RandomForestClassifier(random_state=42), "Random Forest")evaluate_model(MultinomialNB(), "Multinomial Naive Bayes")evaluate_model(HistGradientBoostingClassifier(random_state=42), "Gradient Boosting Classifier")

In [ ]:
# Display the mapping of original labels to encoded labelslabel_mapping = {index: label for index, label in enumerate(label_encoder.classes_)}print("Label Mapping:", label_mapping)